In [1]:
!date

Thu Mar 17 06:56:08 PDT 2022


This notebook uses [Stan](https://mc-stan.org) and [Bean Machine](https://beanmachine.org) to fit a linear model with outliers. The model and data are taken from Jake VanderPlas's post [Frequentism and Bayesianism II: When Results Differ](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/). Specifically, this is [Example #2](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/#Example-#2:-Linear-Fit-with-Outliers) from his post. There's also a very nice discussion of Bayesian modeling for handling outliers in §8.3 of [Data Analysis - A Bayesian Tutorial](https://www.goodreads.com/book/show/1430545.Data_Analysis) by D.S. Sivia. Finally, this discussion is complementary to the official Bean Machine [Robust Linear Regression](https://beanmachine.org/docs/tutorials/#robust-linear-regression) tutorial.

Points are outliers with probability $\theta$. All observations are drawn from a normal distribution. For points that are not outliers, we use the estimated observation error $e_i$ as the standard deviation. For outliers, we use a constant standard deviation $\sigma_o$, which is estimated as part of the model. The model description below, $z$ is a latent indicator for whether or not the point is an outlier.

$$
\begin{align}
\theta & \sim \mathrm{Beta}(2, 5) \\
\sigma_o & \sim \mathrm{Gamma}(2, 2) \\
\beta_0 & \sim \mathcal{N}(0, 10) \\
\beta_1 & \sim \mathcal{N}(0, 10) \\
z & \sim \mathrm{Bernouilli}(\theta) \\
y & \sim \mathcal{N}(\beta_0 + \beta_1 x, \sigma_{z[i]}) \\
\end{align}
$$

where

$$
\sigma_{z[i]} =
    \begin{cases}
    e_i, & z_i = 0 \\
    \sigma_o, & z_i = 1 \\
    \end{cases}
$$

For both the Bean Machine and Stan implementations, we marginalize out the latent discrete variable $z$, as described [here](https://mc-stan.org/docs/2_29/stan-users-guide/summing-out-the-responsibility-parameter.html) in the Stan documentation.

$$
p(y_i | \theta, \beta_0, \beta_1, \sigma_o) = \theta \mathcal{N}(\beta_0 + \beta_1 x_i, \sigma_o) + (1 - \theta) \mathcal{N}(\beta_0 + \beta_1 x_i, e_i)
$$

In [2]:
# This is needed for PyStan to run in a Jupyter notebook
import nest_asyncio

nest_asyncio.apply()


In [3]:
from typing import Union

import altair
import beanmachine.ppl as bm
import numpy as np
import pandas as pd
import stan
import torch
import torch.distributions as dist
from beanmachine.ppl.distributions.unit import Unit
from typing_extensions import TypeGuard


In [4]:
BLUE = "#4e79a7"
ORANGE = "#f28e2b"
RED = "#e15759"
CYAN = "#76b7b2"
GREEN = "#59a14f"
YELLOW = "#edc948"
PURPLE = "#b07aa1"
PINK = "#ff9da7"
BROWN = "#9c755f"
GRAY = "#bab0ac"


In [5]:
def calc_prob_outlier(
    x: torch.Tensor,
    y: torch.Tensor,
    err: torch.Tensor,
    beta_0: float,
    beta_1: float,
    theta: float,
    sigma_o: float,
) -> torch.Tensor:
    """Calculate the probabilities that the points are outliers.

    See https://mc-stan.org/docs/2_29/stan-users-guide/summing-out-the-responsibility-parameter.html#recovering-posterior-mixture-proportions

    Note that I'm computing some sort of best estimate for the probabilities. That is,
    I'm not propagating uncertainty (as represented by samples of the parameters).
    """
    mu = beta_0 + beta_1 * x
    prob_z1 = dist.Normal(mu, err).log_prob(y).exp() * (1 - theta)
    prob_z2 = dist.Normal(mu, sigma_o).log_prob(y).exp() * theta
    return prob_z2 / (prob_z1 + prob_z2)


In [6]:
x_obs = torch.tensor(
    [0, 3, 9, 14, 15, 19, 20, 21, 30, 35, 40, 41, 42, 43, 54, 56, 67, 69, 72, 88],
    dtype=torch.float32,
)
y_obs = torch.tensor(
    [33, 68, 34, 34, 37, 71, 37, 44, 48, 49, 53, 49, 50, 48, 56, 60, 61, 63, 44, 71],
    dtype=torch.float32,
)
err_obs = torch.tensor(
    [
        3.6,
        3.9,
        2.6,
        3.4,
        3.8,
        3.8,
        2.2,
        2.1,
        2.3,
        3.8,
        2.2,
        2.8,
        3.9,
        3.1,
        3.4,
        2.6,
        3.4,
        3.7,
        2.0,
        3.5,
    ]
)


In [7]:
df_obs = pd.DataFrame({"x": x_obs, "y": y_obs, "err": err_obs})


In [8]:
error_bars = (
    altair.Chart(df_obs)
    .mark_errorbar(color=BLUE)
    .encode(altair.X("x"), altair.Y("y"), altair.YError("err"))
)
points = (
    altair.Chart(df_obs)
    .mark_circle(color=BLUE)
    .encode(x=altair.X("x"), y=altair.Y("y", scale=altair.Scale(zero=False)))
)
points + error_bars


alt.LayerChart(...)

In [9]:
STAN_CODE_NAIVE = """
data {
    int<lower=0> n_obs;
    vector[n_obs] x;
    vector[n_obs] y;
    vector[n_obs] err;
}

parameters {
    real beta_0;
    real beta_1;
}

model {
    beta_0 ~ normal(0, 10);
    beta_1 ~ normal(0, 10);

    y ~ normal(beta_0 + beta_1 * x, err);
}
"""


In [10]:
posterior = stan.build(
    program_code=STAN_CODE_NAIVE,
    data={
        "n_obs": len(x_obs),
        "x": x_obs.tolist(),
        "y": y_obs.tolist(),
        "err": err_obs.tolist(),
    },
    random_seed=332211,
)


Building...



Building: found in cache, done.

In [11]:
fit = posterior.sample(num_chains=4, num_samples=1000)


Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 2.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.28 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!


In [12]:
df_samples = fit.to_frame()


In [13]:
df_samples.describe(percentiles=[0.05, 0.5, 0.95]).T


,count,mean,std,min,5%,50%,95%,max
parameters,,,,,,,,
lp__,4000.0,-109.513387,1.050401,-116.919819,-111.616797,-109.188672,-108.519086,-108.472036
accept_stat__,4000.0,0.923421,0.107637,0.251288,0.696539,0.969704,1.000000,1.000000
stepsize__,4000.0,0.422027,0.031133,0.382050,0.382050,0.421039,0.463980,0.463980
treedepth__,4000.0,2.210000,0.720090,1.000000,1.000000,2.000000,3.000000,4.000000
n_leapfrog__,4000.0,6.114500,3.744649,1.000000,1.000000,7.000000,15.000000,15.000000
divergent__,4000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
energy__,4000.0,110.523268,1.444446,108.487704,108.829640,110.203549,113.405848,118.316766
beta_0,4000.0,39.042598,1.249936,34.444524,36.958583,39.059034,41.032308,42.658000
beta_1,4000.0,0.249544,0.028240,0.169302,0.204554,0.248115,0.297175,0.360917


In [14]:
error_bars = (
    altair.Chart(df_obs)
    .mark_errorbar(color=BLUE)
    .encode(altair.X("x"), altair.Y("y"), altair.YError("err"))
)
points = (
    altair.Chart(df_obs)
    .mark_circle(color=BLUE)
    .encode(x=altair.X("x"), y=altair.Y("y", scale=altair.Scale(zero=False)))
)
best_fit = (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: df_samples["beta_0"].mean()
            + df_samples["beta_1"].mean() * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)
points + error_bars + best_fit


alt.LayerChart(...)

In [15]:
STAN_CODE_OUTLIERS = """
data {
    int<lower=0> n_obs;
    vector[n_obs] x;
    vector[n_obs] y;
    vector[n_obs] err;
}

parameters {
    real beta_0;
    real beta_1;
    real<lower=0> sigma_o;
    real<lower=0, upper=1> theta;
}

model {
    real mu;
    beta_0 ~ normal(0, 10);
    beta_1 ~ normal(0, 10);
    sigma_o ~ gamma(2, 2);
    theta ~ beta(2, 5);

    for (i in 1:n_obs) {
        mu = beta_0 + beta_1 * x[i];
        target += log_mix(theta, normal_lpdf(y[i] | mu, sigma_o ), normal_lpdf(y[i] | mu, err[i] ));
    }
}
"""


In [16]:
posterior = stan.build(
    program_code=STAN_CODE_OUTLIERS,
    data={
        "n_obs": len(x_obs),
        "x": x_obs.tolist(),
        "y": y_obs.tolist(),
        "err": err_obs.tolist(),
    },
    random_seed=332211,
)


Building...



Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/9c/th30_62d0lx78835_vdd37sr0000gn/T/httpstan_e528pokx/model_htp43kdw.stan', line 21, column 4: The
    parameter theta is on the left-hand side of more than one tilde
    statement.


In [17]:
fit = posterior.sample(num_chains=4, num_samples=1000)


Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 2.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.23 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.36 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!


In [18]:
df_samples = fit.to_frame()


In [19]:
df_summary = df_samples.describe(percentiles=[0.05, 0.5, 0.95])
df_summary.T


,count,mean,std,min,5%,50%,95%,max
parameters,,,,,,,,
lp__,4000.0,-96.847199,1.506942,-107.462114,-99.743475,-96.512814,-95.069789,-94.743005
accept_stat__,4000.0,0.929085,0.098290,0.136147,0.730945,0.966989,1.000000,1.000000
stepsize__,4000.0,0.337962,0.011483,0.320605,0.320605,0.340878,0.349488,0.349488
treedepth__,4000.0,3.104250,0.693904,1.000000,2.000000,3.000000,4.000000,4.000000
n_leapfrog__,4000.0,10.642000,4.507873,1.000000,3.000000,11.000000,15.000000,31.000000
divergent__,4000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
energy__,4000.0,98.864675,2.049025,94.910001,96.098780,98.561324,102.544776,112.184034
beta_0,4000.0,32.198265,1.863560,25.534571,29.322431,32.081496,35.431268,42.271615
beta_1,4000.0,0.447492,0.041893,0.267811,0.376613,0.448666,0.514474,0.606398


In [20]:
df_params_mean = df_summary.loc["mean"]


In [21]:
prob_outlier = calc_prob_outlier(
    x_obs,
    y_obs,
    err_obs,
    beta_0=df_params_mean["beta_0"],
    beta_1=df_params_mean["beta_1"],
    theta=df_params_mean["theta"],
    sigma_o=df_params_mean["sigma_o"],
)


In [22]:
error_bars = (
    altair.Chart()
    .mark_errorbar()
    .encode(
        altair.X("x"), altair.Y("y"), altair.YError("err"), altair.Color("is_outlier")
    )
)
points = (
    altair.Chart()
    .mark_circle()
    .encode(
        altair.X("x"),
        altair.Y("y", scale=altair.Scale(zero=False)),
        altair.Color("is_outlier"),
    )
)
altair.layer(points, error_bars, data=df_obs.assign(is_outlier=prob_outlier > 0.5),) + (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: df_params_mean["beta_0"] + df_params_mean["beta_1"] * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)


alt.LayerChart(...)

# Bean Machine

In [23]:
@bm.random_variable
def beta_0() -> dist.Distribution:
    return dist.Normal(0, 10)


@bm.random_variable
def beta_1() -> dist.Distribution:
    return dist.Normal(0, 10)


@bm.random_variable
def sigma_o() -> dist.Distribution:
    return dist.Gamma(1, 1)


@bm.random_variable
def theta() -> dist.Distribution:
    return dist.Beta(2, 5)


@bm.random_variable
def y(x_obs: torch.Tensor, y_obs: torch.Tensor) -> dist.Distribution:
    mu = beta_0() + beta_1() * x_obs
    log_likelihood_outlier = torch.log(theta()) + dist.Normal(mu, sigma_o()).log_prob(
        y_obs
    )
    log_likelihood = torch.log(1 - theta()) + dist.Normal(mu, err_obs).log_prob(y_obs)
    return Unit(torch.logaddexp(log_likelihood_outlier, log_likelihood))


In [24]:
altair.Chart(
    pd.DataFrame().assign(
        theta=torch.arange(0, 1, 0.01),
        p=lambda df: dist.Beta(2, 5).log_prob(torch.tensor(df["theta"])).exp(),
    )
).mark_line().encode(x="theta", y="p")


alt.Chart(...)

In [25]:
queries = [beta_0(), beta_1(), sigma_o(), theta()]
observations = {y(x_obs, y_obs): y_obs}


In [26]:
samples = bm.GlobalNoUTurnSampler().infer(
    queries=queries,
    observations=observations,
    num_samples=4000,
    num_adaptive_samples=4000,
    num_chains=4,
)


Samples collected: 100%|██████████| 8000/8000 [02:48<00:00, 47.40it/s] 


In [27]:
df_summary = (
    bm.Diagnostics(samples)
    .summary()
    .set_index(pd.Series(["beta_0", "beta_1", "sigma_o", "theta"]))
)


In [28]:
df_summary


,avg,std,2.5%,50%,97.5%,r_hat,n_eff
beta_0,31.495750,1.693784,28.279526,31.464071,34.946098,1.000913,7109.632812
beta_1,0.462940,0.038920,0.384996,0.463437,0.538241,1.000666,7063.943848
sigma_o,12.818645,1.996976,9.486297,12.621531,17.269719,1.000337,9392.123047
theta,0.271512,0.106472,0.098106,0.260192,0.506772,1.000268,9456.009766


In [29]:
df_params_mean = df_summary["avg"]


In [30]:
prob_outlier = calc_prob_outlier(
    x_obs,
    y_obs,
    err_obs,
    beta_0=df_params_mean["beta_0"],
    beta_1=df_params_mean["beta_1"],
    theta=df_params_mean["theta"],
    sigma_o=df_params_mean["sigma_o"],
)


In [31]:
error_bars = (
    altair.Chart()
    .mark_errorbar()
    .encode(
        altair.X("x"), altair.Y("y"), altair.YError("err"), altair.Color("is_outlier")
    )
)
points = (
    altair.Chart()
    .mark_circle()
    .encode(
        altair.X("x"),
        altair.Y("y", scale=altair.Scale(zero=False)),
        altair.Color("is_outlier"),
    )
)
altair.layer(points, error_bars, data=df_obs.assign(is_outlier=prob_outlier > 0.5),) + (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: samples.get_variable(beta_0()).squeeze().mean().item()
            + samples.get_variable(beta_1()).squeeze().mean().item() * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)


alt.LayerChart(...)

In [32]:
samples = bm.inference.BMGInference().infer(
    queries=queries,
    observations=observations,
    num_samples=4000,
    num_chains=4,
)


ValueError: Fetching the value of attribute log_prob is not supported in Bean Machine Graph.

In [9]:
def is_rvidentifier_list(
    val: list[Union[bm.RVIdentifier, torch.Tensor]]
) -> TypeGuard[list[bm.RVIdentifier]]:
    return all(isinstance(x, bm.RVIdentifier) for x in val)


def is_rvidentifier_dict(
    val: dict[Union[bm.RVIdentifier, torch.Tensor], torch.Tensor]
) -> TypeGuard[dict[bm.RVIdentifier, torch.Tensor]]:
    return all(isinstance(k, bm.RVIdentifier) for k in val)


In [10]:
class LinearRegressionOutliersModel:
    def __init__(
        self, x_obs: torch.Tensor, y_obs: torch.Tensor, err_obs: torch.Tensor
    ) -> None:
        self._x_obs = x_obs
        self._y_obs = y_obs
        self._err_obs = err_obs

    @bm.random_variable
    def beta_0(self) -> dist.Distribution:
        return dist.Normal(0, 10)

    @bm.random_variable
    def beta_1(self) -> dist.Distribution:
        return dist.Normal(0, 10)

    @bm.random_variable
    def sigma_out(self) -> dist.Distribution:
        return dist.Gamma(1, 1)

    @bm.random_variable
    def theta(self) -> dist.Distribution:
        return dist.Beta(2, 5)

    @bm.random_variable
    def z(self, i: int) -> dist.Distribution:
        return dist.Bernoulli(self.theta())

    @bm.random_variable
    def y(self, i: int) -> dist.Distribution:
        return dist.Normal(
            self.beta_0() + self.beta_1() * self._x_obs[i],
            self.z(i) * self.sigma_out() + (1 - self.z(i)) * self._err_obs[i],
        )


In [11]:
model = LinearRegressionOutliersModel(x_obs=x_obs, y_obs=y_obs, err_obs=err_obs)

queries = [model.beta_0(), model.beta_1(), model.sigma_out(), model.theta()]
observations = {model.y(i): y_obs[i] for i in range(len(y_obs))}
assert is_rvidentifier_list(queries)
assert is_rvidentifier_dict(observations)


In [12]:
samples = bm.inference.BMGInference().infer(
    queries=queries,
    observations=observations,
    num_samples=4000,
    num_chains=4,
)

df_summary = (
    bm.Diagnostics(samples)
    .summary()
    .set_index(pd.Series(["beta_0", "beta_1", "sigma_out", "theta"]))
)


In [13]:
df_summary

,avg,std,2.5%,50%,97.5%,r_hat,n_eff
beta_0,31.664923,1.952321,28.335768,31.552090,35.581167,1.004490,842.099609
beta_1,0.458990,0.045576,0.373572,0.461516,0.535678,1.004813,768.476807
sigma_out,12.750916,2.253906,9.205182,12.621003,17.397264,1.003670,1327.304199
theta,0.269679,0.110319,0.089881,0.257363,0.514720,1.000356,5602.959473


In [14]:
df_params_mean = df_summary["avg"]

In [16]:
prob_outlier = calc_prob_outlier(
    x_obs,
    y_obs,
    err_obs,
    beta_0=df_params_mean["beta_0"],
    beta_1=df_params_mean["beta_1"],
    theta=df_params_mean["theta"],
    sigma_o=df_params_mean["sigma_out"],
)

In [22]:
error_bars = (
    altair.Chart()
    .mark_errorbar()
    .encode(
        altair.X("x"), altair.Y("y"), altair.YError("err"), altair.Color("is_outlier")
    )
)
points = (
    altair.Chart()
    .mark_circle()
    .encode(
        altair.X("x"),
        altair.Y("y", scale=altair.Scale(zero=False)),
        altair.Color("is_outlier"),
    )
)
altair.layer(points, error_bars, data=df_obs.assign(is_outlier=prob_outlier > 0.5),) + (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: df_summary.loc["beta_0", "avg"]
            + df_summary.loc["beta_1", "avg"] * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)


alt.LayerChart(...)